## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [1]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [2]:
#Better
!pip install requests BeautifulSoup4 fire

In [74]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys
import pandas as pd
import fire

In [4]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

### Get the most influential twitter users in Africa

In [75]:
def influential(url , tag):
    # scrape the website using beautifulsoup
    data = get_elements(url, tag= tag)
    
    # remove unrelated data from 
    infl_data = data[:-5]
    
    # preprocess the scraped data to remove punctuation marks
    infl_list = []
    marks = '—()'
    for i in infl_data:
        for m in marks:
            i = i.replace(m,"")
        infl_list.append(i.strip(' ').split('@'))
     
    lst = infl_list[-10:][::-1]
    
    # create a dataframe containing the list of influential users 
    df = pd.DataFrame(lst, columns = ['Influencers','Twitter Handle'])
    
    # Save as csv file 
    return df.to_csv('InluencersInAfrica')
    

In [76]:
influential('https://africafreak.com/100-most-influential-twitter-users-in-africa',tag='h2')

### Get The Twitter Handle for African Government Response to covid19

In [79]:
def govern(url, attribute):
    
    # Get the data from the url
    
    response = simple_get(url)
    
    res = get_elements(response, search={'find_all':{'class_':attribute}})
    
    # Get the government twitter handle from the scraped data
    tweet_handle = []

    count = 1
    for index, value in enumerate(res):
        if index == count:
            tweet_handle.append(value)
            count += 3
     
    # preprocess the data to remove punctuation marks
    a = []
    marks = '—()'
    for i in tweet_handle:
        for m in marks:
            i = i.replace(m,"")
        a.append(i.strip(' ').split('@'))
    
    # create a dataframe  and save as a csv file 
    df1 = pd.DataFrame(a, columns = ['Government','Twitter Handle'])
    
    # Get 10 government handles
    df1 = df1[:10]
    return df1.to_csv('GovernmentHandle')

In [80]:
govern('https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa', attribute = 'twitter-tweet')

findaing all of {'class_': 'twitter-tweet'}


## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [15]:
%%bash 

# curl the page and save content to tmp_file
#url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa"
#curl -X GET $url -o tmp_file


#!/bin/bash

# write headers to CSV file
echo "Name, twitter_id" >> extractData.csv
n="1"
while [ $n -lt 2 ]
do
  
  #get title
  title=$(cat tmp_file | grep "class=\"twitter-tweet\"" | cut -d ';' -f1 )
  echo $title
  #get author
  #twitter_id=$(cat tmp_file |grep -A1 "class=\"css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0\"" | tail -1)

  #echo "$title, $twitter_id" >> extractData.csv
  #echo "$title, $twitter_id"
    
  n=$[$n+1]

done

Couldn't find program: 'bash'
